In [0]:
CATALOG = "workspace"
SCHEMA  = "default"
DRY_RUN = False   # <- auf False setzen, um das Löschen auszuführen

spark.sql(f"USE CATALOG `{CATALOG}`")
spark.sql(f"USE SCHEMA `{SCHEMA}`")

def run(sql):
    if DRY_RUN:
        print(sql)
    else:
        spark.sql(sql)

def pick_name(row, prefs=("name","tableName","viewName","function","functionName","volume")):
    d = row.asDict()
    for k in prefs:
        if k in d:
            return d[k]
    return row[1]

In [0]:
# 1) Views droppen
df_views = spark.sql(f"SHOW VIEWS IN `{CATALOG}`.`{SCHEMA}`")
views = [pick_name(r) for r in df_views.collect()]
for v in views:
    run(f"DROP VIEW IF EXISTS `{CATALOG}`.`{SCHEMA}`.`{v}`")

# 2) Tabellen droppen
df_tables = spark.sql(f"SHOW TABLES IN `{CATALOG}`.`{SCHEMA}`")
tables = [pick_name(r) for r in df_tables.collect()]
for t in tables:
    run(f"DROP TABLE IF EXISTS `{CATALOG}`.`{SCHEMA}`.`{t}`")

# 3) User-Functions droppen
df_funcs = spark.sql(f"SHOW USER FUNCTIONS IN `{CATALOG}`.`{SCHEMA}`")
funcs = [pick_name(r) for r in df_funcs.collect()]
for f in funcs:
    run(f"DROP FUNCTION IF EXISTS `{CATALOG}`.`{SCHEMA}`.`{f}`")

# 4) Volumes droppen (am Ende, wenn nichts mehr referenziert)
df_vols = spark.sql(f"SHOW VOLUMES IN `{CATALOG}`.`{SCHEMA}`")
vols = [pick_name(r) for r in df_vols.collect()]
for v in vols:
    run(f"DROP VOLUME IF EXISTS `{CATALOG}`.`{SCHEMA}`.`{v}`")

print("Done. (DRY_RUN =", DRY_RUN, ")")

Done. (DRY_RUN = False )
